In [ ]:
from igraph import *
import random

In [ ]:
def get_permutation(graph):
    permut = []
    for x in range(graph.vcount()):
        permut.append(x)
    random.shuffle(permut)
    random.shuffle(permut)
    return permut

In [ ]:
def get_weight_sum(g):
    sum = 0
    for x in g.es:
        sum += x["weight"]
    return sum

def get_weight_sum_in_community(g,community_num):
    sum = 0
    vertices = g.vs.select(lambda v: v["community"]== community_num)
    for x in vertices:
        for y in vertices:
            sum+=g[x,y]
    return sum

def get_weight_of_links_to_k_in_community(g,k,community_num):
    sum = 0
    vertices = g.vs.select(lambda v: v["community"]== community_num)
    for v in vertices:
        if k!=v.index:
            sum += g[k,v]
    return sum

def get_weight_of_links_to_community(g,community_num,param):
    sum = 0
    vertices = g.vs.select(lambda v: v["community"]== community_num)
    for v in vertices:
        sum += get_weighted_degree(g,v)
    return sum * param

def get_weighted_degree(g,k):
    return g.degree(k,ALL,True)

In [ ]:
def get_weighted_degree_in(g,k):
    sum = 0
    for y in g.neighbors(k):
        sum += g[y,k] 
    return sum

def get_weighted_degree_out(g,k):
    sum = 0
    for y in g.neighbors(k):
        sum += g[k,y] 
    return sum

In [ ]:
def get_modularity_change2(g,i,community,param):    
    value1 = (get_weight_sum_in_community(g,community) + get_weight_of_links_to_k_in_community(g,i,community)*2)/(2*weight_sum(g)) - (((get_weight_of_links_to_community(g,i,community,param) + get_weighted_degree(g,i))/(2*get_weight_sum(g)))**2)
    value2 = (get_weight_sum_in_community(g,community))/(2*get_weight_sum(g)) - (get_weight_of_links_to_community(g,i,community,param)/(2*get_weight_sum(g)))**2 - (get_weighted_degree(g,i)/(2*get_weight_sum(g)))**2
    return value1-value2

def get_modularity_change(g,i,community,degr,m,wol):
    value1= get_weight_of_links_to_k_in_community(g,i,community)/m - (wol*degr)/(2*(m**2))
    return value1


In [ ]:
def change_community(com_table,com_from,com_to):
    i=0
    for x in com_table:
        if x == com_from:
            com_table[i] = com_to
        i += 1
def get_indices(com_table,com):
    l = []
    i = 0
    for x in com_table:
        if x == com:
            l.append(i)
        i += 1
    return l

In [ ]:
def get_vertices_in_community(g,com):
    vertices = g.vs.select(lambda v: v["community"]== com)
    return [v.index for v in vertices]

def get_weight_of_links_between_communities(g,com1,com2):
    vert1 = get_vertices_in_community(g,com1)
    vert2 = get_vertices_in_community(g,com2)
    ret = 0
    if(com1 != com2):
        for v1 in vert1:
            for v2 in vert2:
                ret += g[v1,v2]
    else:
        ret = get_weight_sum_in_community(g,com1)
    return ret

In [ ]:
def merge_communities(g):
    newgraph = Graph()
    for com1 in set(g.vs["community"]):
        newgraph.add_vertex(community =com1)
    i=0
    communities = newgraph.vs["community"]
    for com1 in communities:
        for com2 in communities[i:]:
            weight=get_weight_of_links_between_communities(g,com1,com2)
            if weight>0:
                es = newgraph.add_edge([v["community"] for v in newgraph.vs].index(com1),[v["community"] for v in newgraph.vs].index(com2))
                es["weight"]=weight
        i+=1
        
    return newgraph

In [ ]:
def louvain_iteration(g,permut,com_table,etot):
    changed_table = com_table.copy()
    was_change = False
    m=get_weight_sum(g)
    for x in range(g.vcount()):
        degr = get_weighted_degree(g,permut[x])
        prev_com = g.vs[permut[x]]["community"]
        max_mod_change = 0.0
        max_community = -1
        communities = {}
        for v in g.neighbors(permut[x]):
            if  g.vs[v]["community"] in communities:
                continue
            communities[g.vs[v]["community"]]=1
            g.vs[permut[x]]["community"] = g.vs[v]["community"]

            mod_change = get_modularity_change(g,permut[x],g.vs[v]["community"],degr,m,etot[g.vs[v]["community"]]+degr)

            if(max_mod_change < mod_change):
                max_mod_change = mod_change
                max_community = g.vs[v]["community"]
           
        g.vs[permut[x]]["community"] = prev_com

        if max_community != -1 and max_community != prev_com:
            was_change=True
            g.vs[permut[x]]["community"] = max_community
            etot[prev_com]-=get_weighted_degree(g,permut[x])
            etot[max_community]+=get_weighted_degree(g,permut[x])
            for i in get_indices(com_table,prev_com):
                changed_table[i] = max_community
    com_table[:] = changed_table[:]
    return was_change

In [ ]:
def louvain_algorithm(g,param):
    graph=g.copy()
    for x in range(graph.vcount()):
        graph.vs[x]["community"]=x
    etot = [get_weight_of_links_to_community(graph,com,param) for com in range(graph.vcount())]
    com_table = graph.vs["community"]
    permut = get_permutation(graph)
    flag=True
    while flag:
        flag = louvain_iteration(graph,permut,com_table,etot)
        graph = merge_communities(graph)
        permut = get_permutation(graph)

    return com_table

In [ ]:
def get_modularity(g):
    m = GetWeightSum(g)
    sum = 0
    for i in range(g.vcount()):
        for j in range(g.vcount()):
            if(g.vs[i]["community"] == g.vs[j]["community"]):
                sum += (g[i,j]-((get_weighted_degree(g,i)*get_weighted_degree(g,j))/(2*m)))
    mod = sum /(2*m)
    return mod

In [ ]:
def plot_subgraph(g,vertices):
    subg = g.subgraph(vertices)
 
    com_table=louvain_algorithm(subg)
    for v in subg.vs:
        v["communities"] = []
        v["communities"].append(com_table[v.index])
    plot_full(subg, "full.svg")
    i=0
    for v in subg.vs:
        v["community"] = com_table[i]
        i+=1
    print(get_modularity(subg))

In [ ]:
def get_overlap_percent(indices1, indices2):
    common = list(set(indices1).intersection(indices2))
    if len(indices1) > len(indices2):
        return len(common) / len(indices1)
    else:
        return len(common) / len(indices2)

def assign_communities(original_table, current_table):
    new_table = [0] * len(current_table)
    for x in range(len(current_table)):
        indices = get_indices(current_table,x)
        if len(indices)==0:
            continue
        best_fitting_com=-1
        best_overlap=0
        for y in range(len(original_table)):
            indices2 = get_indices(original_table,y)
            if len(indices2)==0:
                continue
            if get_overlap_percent(indices,indices2)>best_overlap:
                best_overlap=get_overlap_percent(indices,indices2)
                best_fitting_com=y
        if(best_fitting_com != -1):
            indices = get_indices(current_table,x)
            for i in indices:
                new_table[i]=best_fitting_com
    return new_table

In [ ]:
def rearrange_communties(com_table):  
    counter=0
    new_table= com_table.copy()
    for i in range(len(new_table)):
        if i not in new_table:
            for n in range(len(new_table))[i+1:]:
                if n in new_table:
                    indices = get_indices(new_table,n)
                    for x in indices:
                        new_table[x]=i
                    counter += len(indices)
                    break
        if(counter == len(com_table)):
            break
    return new_table

In [ ]:
def update_community_matrix(g,belonging_table,com_table):
    for x in range(g.vcount()):
          for y in g.neighbors(x):
              belonging_table[x][com_table[y]]+=1

In [ ]:
def normalize_belonging(belonging_table):
    for x in range(len(belonging_table)):
        table_sum=sum(belonging_table[x])
        if table_sum ==0:
            continue
        for y in range(len(belonging_table[x])):
            belonging_table[x][y]=belonging_table[x][y]/table_sum
    return belonging_table

In [ ]:
def divide_into_communities(g,ov,belonging_table):
    lambd=1/(ov+1)
    normalize_belonging(belonging_table)
    for x in range(g.vcount()):
            g.vs[x]["communities"]=[]
            if sum(belonging_table[x]) == 0:
                g.vs[x]["communities"].append(x)
                continue
            for y in range(len(belonging_table[x])):
                if belonging_table[x][y]>lambd:
                    g.vs[x]["communities"].append(y+1)

    for x in range(g.vcount()):
        if len( g.vs[x]["communities"])==0:
            i=belonging_table[x].index(max(belonging_table[x]))
            g.vs[x]["communities"].append(i)
            belonging_table[x][i]=1

    for x in range(g.vcount()):
        for y in range(len(belonging_table[x])):
            if belonging_table[x][y] < lambd:
                belonging_table[x][y]=0
    normalize_belonging(belonging_table)
